In [1]:
import requests
import pandas as pd
import os

## Process the classified excel sheets

### BEFORE RUNNING CLOSE ALL THE INVOLVED EXCEL SHEETS

In [27]:
data_dir = "../classification/labelled_data/"
for subdir, dirs, files in os.walk(data_dir):
    for file in files:
        #print os.path.join(subdir, file)
        filepath = subdir + os.sep + file
        
        #print(file)
        print (filepath)

../classification/labelled_data/action\Action_City of Bones_analyzed_reviews.xlsx
../classification/labelled_data/action\Action_Da Vinci Conde_analyzed_reviews.xlsx
../classification/labelled_data/action\Action_Divergent_analyzed_reviews.xlsx
../classification/labelled_data/action\Action_Dune_analyzed_reviews.xlsx
../classification/labelled_data/action\Action_Fight Club_analyzed_reviews.xlsx
../classification/labelled_data/action\Action_Jaws_analyzed_reviews.xlsx
../classification/labelled_data/action\Action_Jurassic Park_analyzed_reviews.xlsx
../classification/labelled_data/action\Action_Last of the Mohicans_analyzed_reviews.xlsx
../classification/labelled_data/action\Action_Ready Player One_analyzed_reviews.xlsx
../classification/labelled_data/action\Action_Starship Troopers_analyzed_reviews.xlsx
../classification/labelled_data/action\Action_The Bourne Identity_analyzed_reviews.xlsx
../classification/labelled_data/action\Action_The Hunger Games_analyzed_reviews.xlsx
../classification

In [28]:
post_dictionary = {}

In [44]:
def convert_time(x):
    temp = x 
    if (isinstance(x,str)):
        temp = pd.to_datetime(temp,format= '%Y-%m-%d %H:%M:%S')
    solr_date = temp.strftime('%Y-%m-%dT%H:%M:%SZ')
    return solr_date

def AddCommentID(row):
    if (row["post_id"] not in post_id_dictionary.keys()):
        post_id_dictionary[row["post_id"]] = 1
    else:
        post_id_dictionary[row["post_id"]] += 1
        
    return str(row["post_id"]) + "_" + str(post_id_dictionary[row["post_id"]])
        
def AddCommentNum(row):
    if (row["post_id"] not in post_id_dictionary.keys()):
        post_id_dictionary[row["post_id"]] = 1
    else:
        post_id_dictionary[row["post_id"]] += 1
        
    return post_id_dictionary[row["post_id"]]

def RenameSentimentColumn(x):
    if "sentiment" in x.lower():
        return "sentiment"
    else:
        return x
    
def RemoveManualLabels(x):
    if ("label" not in x.lower() and "manual" not in x.lower() and "user_sentiment" not in x.lower()):
        return True
    else:
        return False

def ProcessCommentSheet(df, excel_name):
    global post_id_dictionary
    post_id_dictionary = {}
    df["id"] = df.apply(AddCommentID, axis=1)

    # bring id column to the front
    cols = df.columns.tolist()

    # remove manual_label column if it exists
    cols = list(filter(RemoveManualLabels, cols))

    # rename user_sentiment to sentiment
    cols = list(map(RenameSentimentColumn, cols))

    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    
    # add comment num   
    post_id_dictionary = {}
    df["comment_num"] = df.apply(AddCommentNum, axis=1)
    df.to_csv("./sentiment_data/" + excel_name + "_process.csv",index=False)


def ProcessData(subdir, excel_name):
    df = pd.read_excel(subdir + "/" + excel_name)
    
    # print(df.columns.tolist())
    # extract category and append category
    category = excel_name.split('_')[0]
    

    # label TYPE as COMMENT
    type = pd.Series(["COMMENT"] * len(df.index))
    df["TYPE"] = type   

    category = pd.Series([category] * len(df.index))
    df["category"] = category
    
    # Convert time into pdate format as required by Solr
    created_utc = df['created_utc']
    created_utc = created_utc.apply(convert_time)
    df['created_utc'] = created_utc

    excel_name_separated = excel_name.split('_')

    bk_name = pd.Series([excel_name_separated[1]] * len(df.index))
    df['book'] = bk_name

    new_sheet_name = excel_name_separated[1] + "_" + excel_name_separated[2]

    ProcessCommentSheet(df, new_sheet_name)
    

    return df

In [45]:
ProcessData("../classification/labelled_data/horror", "Horror_Frankenstein_analyzed_reviews.xlsx")

,post_id,comment_text,author,created_utc,sentiment,manual_label,TYPE,category,book,id
0,z3aqav,I read Paradise Lost before Frankenstein and I...,Zoravor,2022-11-24T08:29:37Z,2,1.0,COMMENT,Horror,Frankenstein,z3aqav_1
1,z3aqav,"Oh god yes, Shelly's Frankenstein is a masterp...",Zygomatical,2022-11-24T05:26:18Z,1,1.0,COMMENT,Horror,Frankenstein,z3aqav_2
2,z3aqav,Frankenstein and Dracula blew my mind. Been tr...,OakendaleAbbey,2022-11-24T10:13:43Z,1,1.0,COMMENT,Horror,Frankenstein,z3aqav_3
3,z3aqav,Would you suggest any of those other ones for ...,CantankerousOctopus,2022-11-24T06:56:33Z,2,0.0,COMMENT,Horror,Frankenstein,z3aqav_4
4,z3aqav,The funny thing is that everyone expects they'...,NaN,2022-11-24T05:59:24Z,3,3.0,COMMENT,Horror,Frankenstein,z3aqav_5
...,...,...,...,...,...,...,...,...,...,...
1875,95eq1,"If you're PDF-allergic, [here's the HTML versi...",Captain_Midnight,2009-07-28T23:24:20Z,3,NaN,COMMENT,Horror,Frankenstein,95eq1_1
1876,6do0v0,"Thanks for your questions! In the ""Professor N...",SimonThalmann,2017-05-27T15:42:29Z,2,NaN,COMMENT,Horror,Frankenstein,6do0v0_1
1877,6do0v0,"Great questions, thank you! As a full-time dig...",SimonThalmann,2017-05-27T22:21:02Z,2,NaN,COMMENT,Horror,Frankenstein,6do0v0_2
1878,mwhxt,"Uh. From the book's website, it sounds *horrif...",maxd,2011-12-02T23:56:37Z,0,NaN,COMMENT,Horror,Frankenstein,mwhxt_1


In [40]:
data_dir = "../classification/labelled_data/"
for subdir, dirs, files in os.walk(data_dir):
    for file in files:
        #print os.path.join(subdir, file)
        filepath = subdir + os.sep + file
        print(file)
        ProcessData(subdir, file)

Action_City of Bones_analyzed_reviews.xlsx
Action_Da Vinci Conde_analyzed_reviews.xlsx
Action_Divergent_analyzed_reviews.xlsx
Action_Dune_analyzed_reviews.xlsx


C:\Users\Dayou\AppData\Local\Temp\ipykernel_12048\806283518.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["comment_num"] = df.apply(AddCommentNum, axis=1)
C:\Users\Dayou\AppData\Local\Temp\ipykernel_12048\806283518.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["comment_num"] = df.apply(AddCommentNum, axis=1)
C:\Users\Dayou\AppData\Local\Temp\ipykernel_12048\806283518.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

Action_Fight Club_analyzed_reviews.xlsx
Action_Jaws_analyzed_reviews.xlsx
Action_Jurassic Park_analyzed_reviews.xlsx
Action_Last of the Mohicans_analyzed_reviews.xlsx
Action_Ready Player One_analyzed_reviews.xlsx
Action_Starship Troopers_analyzed_reviews.xlsx
Action_The Bourne Identity_analyzed_reviews.xlsx
Action_The Hunger Games_analyzed_reviews.xlsx


C:\Users\Dayou\AppData\Local\Temp\ipykernel_12048\806283518.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["comment_num"] = df.apply(AddCommentNum, axis=1)
C:\Users\Dayou\AppData\Local\Temp\ipykernel_12048\806283518.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["comment_num"] = df.apply(AddCommentNum, axis=1)
C:\Users\Dayou\AppData\Local\Temp\ipykernel_12048\806283518.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

Action_The Martian_analyzed_reviews.xlsx
Action_The Maze Runner_analyzed_reviews.xlsx


...

C:\Users\Dayou\AppData\Local\Temp\ipykernel_12048\806283518.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["comment_num"] = df.apply(AddCommentNum, axis=1)
C:\Users\Dayou\AppData\Local\Temp\ipykernel_12048\806283518.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["comment_num"] = df.apply(AddCommentNum, axis=1)
C:\Users\Dayou\AppData\Local\Temp\ipykernel_12048\806283518.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

Mystery_Mystic River_analyzed_reviews.xlsx
Mystery_Rebecca_analyzed_reviews.xlsx
Mystery_Red Dragon_analyzed_reviews.xlsx
Mystery_Shutter Island_analyzed_reviews.xlsx
Mystery_The Alienist_analyzed_reviews.xlsx
Mystery_The Bone Collector_analyzed_reviews.xlsx
Mystery_The Firm_analyzed_reviews.xlsx
Mystery_The Godfather_analyzed_reviews.xlsx
Mystery_The Maid_analyzed_reviews.xlsx
Mystery_The Moonstone_analyzed_reviews.xlsx


C:\Users\Dayou\AppData\Local\Temp\ipykernel_12048\806283518.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["comment_num"] = df.apply(AddCommentNum, axis=1)
C:\Users\Dayou\AppData\Local\Temp\ipykernel_12048\806283518.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["comment_num"] = df.apply(AddCommentNum, axis=1)
C:\Users\Dayou\AppData\Local\Temp\ipykernel_12048\806283518.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

Mystery_The Secret History_analyzed_reviews.xlsx
Mystery_The Thirteenth Tale_analyzed_reviews.xlsx


C:\Users\Dayou\AppData\Local\Temp\ipykernel_12048\806283518.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["comment_num"] = df.apply(AddCommentNum, axis=1)
